# Understanding the Agent Loop

**Deep Dive into How Agents Think and Act**

---

Welcome to an in-depth exploration of the **Strands Agent Loop**! This notebook reveals the inner workings of how agents process information, make decisions, and generate responses. By the end of this tutorial, you'll understand the elegant architecture that powers intelligent agent behavior.

### 🎯 What You'll Learn

In this technical deep-dive, you will:
- Understand the core components of the agent loop
- Trace how messages flow through the system
- See how agents decide when to use tools
- Learn about recursive reasoning and multi-step workflows
- Debug and monitor agent decision-making
- Build intuition for creating more sophisticated agents

### 🔄 What is the Agent Loop?

The **Agent Loop** is the heart of every Strands agent. It's a sophisticated cycle that:
1. **Receives** user input
2. **Processes** it with an AI model
3. **Decides** on actions (including tool use)
4. **Executes** those actions
5. **Reasons** about results
6. **Generates** responses

This loop can iterate multiple times in a single interaction, enabling complex reasoning!

## 📦 Step 1: Setup and Imports

### Installing Dependencies
We'll need the core Strands SDK and some example tools to demonstrate the agent loop in action.

In [2]:
# Install required packages
%pip install strands-agents strands-agents-tools -q

# Import necessary modules
import json
import time
import logging
from strands import Agent, tool
from strands_tools import calculator, current_time

print("✅ Setup complete! Ready to explore the agent loop.")

Note: you may need to restart the kernel to use updated packages.
✅ Setup complete! Ready to explore the agent loop.


## 🔍 Step 2: Enabling Debug Logging

### See What's Happening Under the Hood
To understand the agent loop, we need to see what's happening internally. Let's enable debug logging to trace the agent's thought process.

In [4]:
# 🚀 EASIEST OPTION: Paste your Bedrock API Key here
# Get your API key from: https://console.aws.amazon.com/bedrock/ -> API keys
API_KEY = ""  # Paste your API key between the quotes

# Configuration constants (you can modify these if needed)
REGION_NAME = "us-west-2"
AWS_PROFILE = "default"

# Set up authentication using our utility function
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent / "src"))  # Add the src directory to path for imports
from auth_utils import setup_bedrock_auth, display_auth_status

auth_status = setup_bedrock_auth(
    api_key=API_KEY,
    region_name=REGION_NAME,
    aws_profile=AWS_PROFILE
)

# Display the authentication status
display_auth_status(auth_status)

# Configure logging to see agent internals
logging.getLogger("strands").setLevel(logging.DEBUG)
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s",
    handlers=[logging.StreamHandler()],
    level=logging.INFO
)

print("\n🔍 Debug logging enabled!")
print("   You'll now see detailed information about the agent's internal processes.")

🔄 No API key provided, using traditional AWS authentication
   Checking for AWS credentials...
✅ AWS credentials found!

✅ Authentication Method: AWS Profile
   Using: AWS credentials file
   Region: us-west-2 (Profile: default)
💡 Ready to create agents!

🔍 Debug logging enabled!
   You'll now see detailed information about the agent's internal processes.


## 🔧 Step 3: The Tool-Using Agent Loop

### Multi-Step Reasoning
Let's see how the agent loop handles tool usage. When an agent has tools available, the loop becomes more complex:

1. **User Input** → 2. **Model Processing** → 3. **Tool Request** → 4. **Tool Execution** → 5. **Result Processing** → 6. **Final Response**

In [6]:
# Create an agent with tools
tool_agent = Agent(
    model=auth_status.strands_bedrock_model,
    tools=[calculator, current_time],
    system_prompt="You are a helpful assistant with access to tools. Use them when needed."
)

print("🔧 TOOL-USING AGENT LOOP DEMONSTRATION")
print("=" * 50)
print("\n📍 Watch how the agent loop handles tool usage:\n")

# Send a message that requires tool use
start_time = time.time()
response = tool_agent("What is 25 * 48? Also, what time is it?")
end_time = time.time()

print(f"⏱️  Total Processing Time: {end_time - start_time:.2f} seconds")

# Analyze the conversation history
print("\n📊 Agent Loop Analysis:")
print("=" * 50)
for i, msg in enumerate(tool_agent.messages):
    role = msg.get('role', 'unknown')
    content = msg.get('content', [])
    
    print(f"\n🔄 Loop Step {i+1} - Role: {role}")
    
    for item in content:
        if 'text' in item:
            print(f"   💬 Text: {item['text'][:100]}...")
        elif 'toolUse' in item:
            tool_use = item['toolUse']
            print(f"   🔧 Tool Request: {tool_use.get('name')}")
            print(f"      Input: {tool_use.get('input')}")
        elif 'toolResult' in item:
            tool_result = item['toolResult']
            print(f"   ✅ Tool Result: {tool_result.get('content', [{}])[0].get('text', 'N/A')}")

DEBUG | strands.models.bedrock | config=<{'model_id': 'us.anthropic.claude-3-7-sonnet-20250219-v1:0'}> | initializing
DEBUG | strands.tools.registry | tool_name=<calculator>, tool_type=<function>, is_dynamic=<False> | registering tool
DEBUG | strands.tools.registry | tool_name=<current_time>, tool_type=<function>, is_dynamic=<False> | registering tool
DEBUG | strands.tools.registry | tools_dir=<d:\Code Workspace\strands\notebooks\tools> | tools directory not found
DEBUG | strands.tools.registry | tool_modules=<[]> | discovered
DEBUG | strands.tools.registry | tool_count=<0>, success_count=<0> | finished loading tools
DEBUG | strands.tools.registry | tools_dir=<d:\Code Workspace\strands\notebooks\tools> | tools directory not found
DEBUG | strands.tools.registry | getting tool configurations
DEBUG | strands.tools.registry | tool_name=<calculator> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<current_time> | loaded tool config
DEBUG | strands.tools.registry | tool_count

🔧 TOOL-USING AGENT LOOP DEMONSTRATION

📍 Watch how the agent loop handles tool usage:

I'll help you with both of these questions using the tools I have available.

First, let me calculate 25 * 48:
Tool #2: calculator


DEBUG | strands.types.models.model | finished streaming response from model
DEBUG | strands.tools.executor | tool_count=<1>, tool_executor=<ThreadPoolExecutorWrapper> | executing tools in parallel
DEBUG | strands.handlers.tool_handler | tool=<{'toolUseId': 'tooluse_7E2s_XYMRtyriIGttwOp9w', 'name': 'calculator', 'input': {'expression': '25 * 48', 'mode': 'evaluate'}}> | invoking
DEBUG | strands.tools.executor | tool_count=<1> | submitted tasks to parallel executor


╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭───────────┬─────────────────────╮                                                                            │
│  │ Operation │ Evaluate Expression │                                                                            │
│  │ Input     │ 25 * 48             │                                                                            │
│  │ Result    │ 1200                │                                                                            │
│  ╰───────────┴─────────────────────╯                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

DEBUG | strands.event_loop.streaming | model=<<strands.models.bedrock.BedrockModel object at 0x0000023658869A90>> | streaming messages
DEBUG | strands.types.models.model | formatting request
DEBUG | strands.types.models.model | invoking model
DEBUG | strands.types.models.model | got response from model


Now, let me check the current time:

DEBUG | strands.types.models.model | finished streaming response from model
DEBUG | strands.handlers.tool_handler | tool=<{'toolUseId': 'tooluse_4bVkaMOqQDC-YGYU_O5HJg', 'name': 'current_time', 'input': {}}> | invoking
DEBUG | strands.event_loop.streaming | model=<<strands.models.bedrock.BedrockModel object at 0x0000023658869A90>> | streaming messages
DEBUG | strands.types.models.model | formatting request
DEBUG | strands.types.models.model | invoking model
DEBUG | strands.types.models.model | got response from model



Tool #3: current_time
25 × 48 = 1200

The current time is 2025-09-10T08:08:26.908302+00:00 

DEBUG | strands.types.models.model | finished streaming response from model
DEBUG | strands.agent.conversation_manager.sliding_window_conversation_manager | window_size=<6>, message_count=<40> | skipping context reduction


(UTC time format).⏱️  Total Processing Time: 9.89 seconds

📊 Agent Loop Analysis:

🔄 Loop Step 1 - Role: user
   💬 Text: What is 25 * 48? Also, what time is it?...

🔄 Loop Step 2 - Role: assistant
   💬 Text: I'll help you with both of these questions using the tools I have available.

First, let me calculat...
   🔧 Tool Request: calculator
      Input: {'expression': '25 * 48', 'mode': 'evaluate'}

🔄 Loop Step 3 - Role: user
   ✅ Tool Result: Result: 1200

🔄 Loop Step 4 - Role: assistant
   💬 Text: Now, let me check the current time:...
   🔧 Tool Request: current_time
      Input: {}

🔄 Loop Step 5 - Role: user
   ✅ Tool Result: 2025-09-10T08:08:26.908302+00:00

🔄 Loop Step 6 - Role: assistant
   💬 Text: 25 × 48 = 1200

The current time is 2025-09-10T08:08:26.908302+00:00 (UTC time format)....


## 🌀 Step 4: Recursive Agent Loops

### Complex Multi-Step Reasoning
The agent loop can recursively call itself when multiple tool uses are needed. Let's create a scenario that demonstrates this recursive behavior.

In [7]:
# Create custom tools to demonstrate recursive loops
@tool
def get_word_count(text: str) -> int:
    """Count the number of words in a text."""
    return len(text.split())

@tool
def get_character_count(text: str) -> int:
    """Count the number of characters in a text."""
    return len(text)

@tool
def calculate_reading_time(word_count: int) -> float:
    """Calculate estimated reading time in minutes (200 words per minute)."""
    return round(word_count / 200, 2)

# Create an agent with these tools
recursive_agent = Agent(
    model=auth_status.strands_bedrock_model,
    tools=[get_word_count, get_character_count, calculate_reading_time],
    system_prompt="You are a text analysis assistant. Analyze text thoroughly using available tools."
)

print("🌀 RECURSIVE AGENT LOOP DEMONSTRATION")
print("=" * 50)
print("\n📍 Watch how the agent makes multiple tool calls:\n")

# Text to analyze
sample_text = """The agent loop is a fascinating concept in AI. It allows agents to reason 
step by step, use tools when needed, and build complex responses through iterative processing."""

# Send the analysis request
response = recursive_agent(f"Analyze this text and give me all statistics: '{sample_text}'")

# Count the number of tool uses
tool_uses = 0
for msg in recursive_agent.messages:
    for item in msg.get('content', []):
        if 'toolUse' in item:
            tool_uses += 1

print(f"\n📊 Loop Statistics:")
print(f"   Total messages: {len(recursive_agent.messages)}")
print(f"   Tool uses: {tool_uses}")
print(f"   Loop iterations: {(len(recursive_agent.messages) - 1) // 2}")

DEBUG | strands.models.bedrock | config=<{'model_id': 'us.anthropic.claude-3-7-sonnet-20250219-v1:0'}> | initializing
DEBUG | strands.tools.registry | tool_name=<get_word_count> | registering function tool
DEBUG | strands.tools.registry | tool_name=<get_word_count>, tool_type=<function>, is_dynamic=<False> | registering tool
DEBUG | strands.tools.registry | tool_name=<get_character_count> | registering function tool
DEBUG | strands.tools.registry | tool_name=<get_character_count>, tool_type=<function>, is_dynamic=<False> | registering tool
DEBUG | strands.tools.registry | tool_name=<calculate_reading_time> | registering function tool
DEBUG | strands.tools.registry | tool_name=<calculate_reading_time>, tool_type=<function>, is_dynamic=<False> | registering tool
DEBUG | strands.tools.registry | tools_dir=<d:\Code Workspace\strands\notebooks\tools> | tools directory not found
DEBUG | strands.tools.registry | tool_modules=<[]> | discovered
DEBUG | strands.tools.registry | tool_count=<0>, s

🌀 RECURSIVE AGENT LOOP DEMONSTRATION

📍 Watch how the agent makes multiple tool calls:

I'll analyze the provided text using the available tools to give you statistics.
Tool #4: get_word_count


DEBUG | strands.types.models.model | finished streaming response from model
DEBUG | strands.tools.executor | tool_count=<1>, tool_executor=<ThreadPoolExecutorWrapper> | executing tools in parallel
DEBUG | strands.handlers.tool_handler | tool=<{'toolUseId': 'tooluse_7zfcPb82RhW1DSqXIFKyiw', 'name': 'get_word_count', 'input': {'text': 'The agent loop is a fascinating concept in AI. It allows agents to reason \nstep by step, use tools when needed, and build complex responses through iterative processing.'}}> | invoking
DEBUG | strands.tools.executor | tool_count=<1> | submitted tasks to parallel executor
DEBUG | strands.event_loop.streaming | model=<<strands.models.bedrock.BedrockModel object at 0x0000023658976C40>> | streaming messages
DEBUG | strands.types.models.model | formatting request
DEBUG | strands.types.models.model | invoking model
DEBUG | strands.types.models.model | got response from model



Tool #5: get_character_count


DEBUG | strands.types.models.model | finished streaming response from model
DEBUG | strands.handlers.tool_handler | tool=<{'toolUseId': 'tooluse_Ali9tpVXRSWQUm4xXij62g', 'name': 'get_character_count', 'input': {'text': 'The agent loop is a fascinating concept in AI. It allows agents to reason \nstep by step, use tools when needed, and build complex responses through iterative processing.'}}> | invoking
DEBUG | strands.event_loop.streaming | model=<<strands.models.bedrock.BedrockModel object at 0x0000023658976C40>> | streaming messages
DEBUG | strands.types.models.model | formatting request
DEBUG | strands.types.models.model | invoking model
DEBUG | strands.types.models.model | got response from model



Tool #6: calculate_reading_time


DEBUG | strands.types.models.model | finished streaming response from model
DEBUG | strands.handlers.tool_handler | tool=<{'toolUseId': 'tooluse_NG__JdPtRxaPM208tqyYjA', 'name': 'calculate_reading_time', 'input': {'word_count': 28}}> | invoking
DEBUG | strands.event_loop.streaming | model=<<strands.models.bedrock.BedrockModel object at 0x0000023658976C40>> | streaming messages
DEBUG | strands.types.models.model | formatting request
DEBUG | strands.types.models.model | invoking model
DEBUG | strands.types.models.model | got response from model


## Text Analysis Statistics

Here are all the statistics for the provided text:

1. **Word Count**: 28 words
2. **Character Count**: 169 characters
3. **Estimated Reading Time**: 0.14 minutes (about 8.4 seconds)

The text is a brief description of the agent loop concept in AI, explaining how it enables agents to work through problems methodically and utilize tools as

DEBUG | strands.types.models.model | finished streaming response from model
DEBUG | strands.agent.conversation_manager.sliding_window_conversation_manager | window_size=<8>, message_count=<40> | skipping context reduction


 needed.
📊 Loop Statistics:
   Total messages: 8
   Tool uses: 3
   Loop iterations: 3


## 🎯 Step 5: Tracing Agent Decisions

### Understanding Why Agents Make Choices
Let's create a custom callback handler to trace exactly what happens at each step of the agent loop. This gives us deep insights into the decision-making process.

In [8]:
# Create a detailed callback handler
class AgentLoopTracer:
    def __init__(self):
        self.events = []
        self.current_step = 0
    
    def __call__(self, **kwargs):
        self.current_step += 1
        event = {
            'step': self.current_step,
            'timestamp': time.time(),
            'type': 'unknown'
        }
        
        if 'data' in kwargs:
            event['type'] = 'text_generation'
            event['data'] = kwargs['data'][:50] + '...' if len(kwargs['data']) > 50 else kwargs['data']
        elif 'current_tool_use' in kwargs:
            event['type'] = 'tool_decision'
            event['tool'] = kwargs['current_tool_use'].get('name')
        elif 'event_type' in kwargs:
            event['type'] = kwargs['event_type']
            
        self.events.append(event)
    
    def print_trace(self):
        print("\n🔍 AGENT LOOP TRACE:")
        print("=" * 50)
        start_time = self.events[0]['timestamp'] if self.events else 0
        
        for event in self.events:
            elapsed = event['timestamp'] - start_time
            print(f"\n⏱️  +{elapsed:.3f}s - Step {event['step']}")
            print(f"   Type: {event['type']}")
            
            if event['type'] == 'text_generation':
                print(f"   Generated: {event.get('data', 'N/A')}")
            elif event['type'] == 'tool_decision':
                print(f"   Tool: {event.get('tool', 'N/A')}")

# Create tracer and agent
tracer = AgentLoopTracer()
traced_agent = Agent(
    model=auth_status.strands_bedrock_model,
    tools=[calculator, current_time],
    callback_handler=tracer
)

print("🎯 TRACING AGENT DECISIONS")
print("=" * 50)

# Execute a complex request
response = traced_agent(
    "Calculate 156 * 89, then tell me what time it is. "
    "Finally, calculate how many seconds are in the result of the first calculation."
)

# Print the trace
tracer.print_trace()

print(f"\n\n🤖 Final Response: {response}")

DEBUG | strands.models.bedrock | config=<{'model_id': 'us.anthropic.claude-3-7-sonnet-20250219-v1:0'}> | initializing
DEBUG | strands.tools.registry | tool_name=<calculator>, tool_type=<function>, is_dynamic=<False> | registering tool
DEBUG | strands.tools.registry | tool_name=<current_time>, tool_type=<function>, is_dynamic=<False> | registering tool
DEBUG | strands.tools.registry | tools_dir=<d:\Code Workspace\strands\notebooks\tools> | tools directory not found
DEBUG | strands.tools.registry | tool_modules=<[]> | discovered
DEBUG | strands.tools.registry | tool_count=<0>, success_count=<0> | finished loading tools
DEBUG | strands.tools.registry | tools_dir=<d:\Code Workspace\strands\notebooks\tools> | tools directory not found
DEBUG | strands.tools.registry | getting tool configurations
DEBUG | strands.tools.registry | tool_name=<calculator> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<current_time> | loaded tool config
DEBUG | strands.tools.registry | tool_count

🎯 TRACING AGENT DECISIONS


DEBUG | strands.types.models.model | finished streaming response from model
DEBUG | strands.tools.executor | tool_count=<1>, tool_executor=<ThreadPoolExecutorWrapper> | executing tools in parallel
DEBUG | strands.handlers.tool_handler | tool=<{'toolUseId': 'tooluse_e6wtu5ggQr6J76OQoKvsig', 'name': 'calculator', 'input': {'expression': '156 * 89', 'mode': 'evaluate'}}> | invoking
DEBUG | strands.tools.executor | tool_count=<1> | submitted tasks to parallel executor


╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭───────────┬─────────────────────╮                                                                            │
│  │ Operation │ Evaluate Expression │                                                                            │
│  │ Input     │ 156 * 89            │                                                                            │
│  │ Result    │ 13884               │                                                                            │
│  ╰───────────┴─────────────────────╯                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

DEBUG | strands.event_loop.streaming | model=<<strands.models.bedrock.BedrockModel object at 0x0000023658CCD0F0>> | streaming messages
DEBUG | strands.types.models.model | formatting request
DEBUG | strands.types.models.model | invoking model
DEBUG | strands.types.models.model | got response from model
DEBUG | strands.types.models.model | finished streaming response from model
DEBUG | strands.handlers.tool_handler | tool=<{'toolUseId': 'tooluse_FCeQbmcYTIKi_C-SeJLNKw', 'name': 'current_time', 'input': {}}> | invoking
DEBUG | strands.event_loop.streaming | model=<<strands.models.bedrock.BedrockModel object at 0x0000023658CCD0F0>> | streaming messages
DEBUG | strands.types.models.model | formatting request
DEBUG | strands.types.models.model | invoking model
DEBUG | strands.types.models.model | got response from model
DEBUG | strands.types.models.model | finished streaming response from model
DEBUG | strands.handlers.tool_handler | tool=<{'toolUseId': 'tooluse_eg5RYTDaTSWsrzTb_UeezQ', 'na

╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭───────────┬─────────────────────╮                                                                            │
│  │ Operation │ Evaluate Expression │                                                                            │
│  │ Input     │ 13884 * 1           │                                                                            │
│  │ Result    │ 13884               │                                                                            │
│  ╰───────────┴─────────────────────╯                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

DEBUG | strands.event_loop.streaming | model=<<strands.models.bedrock.BedrockModel object at 0x0000023658CCD0F0>> | streaming messages
DEBUG | strands.types.models.model | formatting request
DEBUG | strands.types.models.model | invoking model
DEBUG | strands.types.models.model | got response from model
DEBUG | strands.types.models.model | finished streaming response from model
DEBUG | strands.agent.conversation_manager.sliding_window_conversation_manager | window_size=<8>, message_count=<40> | skipping context reduction



🔍 AGENT LOOP TRACE:

⏱️  +0.000s - Step 1
   Type: unknown

⏱️  +0.003s - Step 2
   Type: unknown

⏱️  +0.003s - Step 3
   Type: unknown

⏱️  +2.807s - Step 4
   Type: unknown

⏱️  +2.807s - Step 5
   Type: unknown

⏱️  +2.807s - Step 6
   Type: text_generation
   Generated: I

⏱️  +2.807s - Step 7
   Type: unknown

⏱️  +2.807s - Step 8
   Type: text_generation
   Generated: 'll solve this step by

⏱️  +2.807s - Step 9
   Type: unknown

⏱️  +2.807s - Step 10
   Type: text_generation
   Generated:  step using the available tools.

⏱️  +2.807s - Step 11
   Type: unknown

⏱️  +2.807s - Step 12
   Type: text_generation
   Generated: 

First, let's calculate

⏱️  +2.807s - Step 13
   Type: unknown

⏱️  +2.807s - Step 14
   Type: text_generation
   Generated:  156 * 89:

⏱️  +3.218s - Step 15
   Type: unknown

⏱️  +3.218s - Step 16
   Type: unknown

⏱️  +3.218s - Step 17
   Type: unknown

⏱️  +3.218s - Step 18
   Type: tool_decision
   Tool: calculator

⏱️  +4.575s - Step 19
   Type: unknow

## 🔄 Step 6: Parallel Tool Execution

### Optimizing the Agent Loop
Strands agents can execute multiple tools in parallel when they're independent. This optimization in the agent loop significantly improves performance.

In [5]:
# Create tools that simulate longer operations
@tool
def slow_calculation(x: int, y: int) -> int:
    """Perform a slow calculation (simulated)."""
    time.sleep(5)  # Simulate processing time
    return x + y

@tool
def slow_multiplication(x: int, y: int) -> int:
    """Perform a slow multiplication (simulated)."""
    time.sleep(5)  # Simulate processing time
    return x * y
# Create a common system prompt that encourages parallel execution
system_prompt="When given multiple independent tasks, always make ALL tool calls in a single response to enable parallel execution."

# Create agents with different parallel settings
sequential_agent = Agent(
    system_prompt=system_prompt,
    model=auth_status.strands_bedrock_model,
    tools=[slow_calculation, slow_multiplication],
    max_parallel_tools=1  # Sequential execution
)

parallel_agent = Agent(
    system_prompt=system_prompt,
    model=auth_status.strands_bedrock_model,
    tools=[slow_calculation, slow_multiplication],
    max_parallel_tools=4  # Parallel execution
)

print("🔄 PARALLEL VS SEQUENTIAL EXECUTION")
print("=" * 50)

# Test sequential execution
print("\n1️⃣ Sequential Execution (max_parallel_tools=1):")
prompt = "I need you to make TWO tool calls in your response: call slow_calculation with x=10, y=20 AND call slow_multiplication with x=5, y=6. Make both calls at once."
start_time = time.time()
seq_response = sequential_agent(
    prompt=prompt
)
seq_time = time.time() - start_time
print(f"   Response: {seq_response}")
print(f"   ⏱️  Time: {seq_time:.2f} seconds")

# Test parallel execution
print("\n2️⃣ Parallel Execution (max_parallel_tools=4):")
start_time = time.time()
par_response = parallel_agent(
    prompt=prompt
)
par_time = time.time() - start_time
print(f"   Response: {par_response}")
print(f"   ⏱️  Time: {par_time:.2f} seconds")

print(f"\n⚡ Performance Improvement: {seq_time/par_time:.1f}x faster with parallel execution!")

DEBUG | strands.models.bedrock | config=<{'model_id': 'us.anthropic.claude-3-7-sonnet-20250219-v1:0'}> | initializing
DEBUG | strands.tools.registry | tool_name=<slow_calculation> | registering function tool
DEBUG | strands.tools.registry | tool_name=<slow_calculation>, tool_type=<function>, is_dynamic=<False> | registering tool
DEBUG | strands.tools.registry | tool_name=<slow_multiplication> | registering function tool
DEBUG | strands.tools.registry | tool_name=<slow_multiplication>, tool_type=<function>, is_dynamic=<False> | registering tool
DEBUG | strands.tools.registry | tools_dir=<d:\Code Workspace\strands\notebooks\tools> | tools directory not found
DEBUG | strands.tools.registry | tool_modules=<[]> | discovered
DEBUG | strands.tools.registry | tool_count=<0>, success_count=<0> | finished loading tools
DEBUG | strands.tools.registry | tools_dir=<d:\Code Workspace\strands\notebooks\tools> | tools directory not found
DEBUG | strands.tools.watcher | tool directory watching initiali

🔄 PARALLEL VS SEQUENTIAL EXECUTION

1️⃣ Sequential Execution (max_parallel_tools=1):
I'll make both tool calls at once for you.
Tool #1: slow_calculation

Tool #2: slow_multiplication


DEBUG | strands.types.models.model | finished streaming response from model
DEBUG | strands.handlers.tool_handler | tool=<{'toolUseId': 'tooluse_THokoCxRQTiHiP3v2lZJIg', 'name': 'slow_calculation', 'input': {'x': 10, 'y': 20}}> | invoking
DEBUG | strands.handlers.tool_handler | tool=<{'toolUseId': 'tooluse_k1_7RoUFRDWBnJpnJnbW1Q', 'name': 'slow_multiplication', 'input': {'x': 5, 'y': 6}}> | invoking
DEBUG | strands.event_loop.streaming | model=<<strands.models.bedrock.BedrockModel object at 0x000002209B926CF0>> | streaming messages
DEBUG | strands.types.models.model | formatting request
DEBUG | strands.types.models.model | invoking model
DEBUG | strands.types.models.model | got response from model


I've completed both calculations for you:

1. slow_calculation with x=10 and y=20 resulted in: 30
2. slow_multiplication with x=5 and y=6 resulted in: 30

Both operations were performed simultaneously to save time.

DEBUG | strands.types.models.model | finished streaming response from model
DEBUG | strands.agent.conversation_manager.sliding_window_conversation_manager | window_size=<4>, message_count=<40> | skipping context reduction
DEBUG | strands.tools.registry | getting tool configurations
DEBUG | strands.tools.registry | tool_name=<slow_calculation> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<slow_multiplication> | loaded tool config
DEBUG | strands.tools.registry | tool_count=<2> | tools configured
DEBUG | strands.tools.registry | getting tool configurations
DEBUG | strands.tools.registry | tool_name=<slow_calculation> | loaded tool config
DEBUG | strands.tools.registry | tool_name=<slow_multiplication> | loaded tool config
DEBUG | strands.tools.registry | tool_count=<2> | tools configured
DEBUG | strands.event_loop.streaming | model=<<strands.models.bedrock.BedrockModel object at 0x000002209BB5A490>> | streaming messages
DEBUG | strands.types.models.model | formatting r

   Response: I've completed both calculations for you:

1. slow_calculation with x=10 and y=20 resulted in: 30
2. slow_multiplication with x=5 and y=6 resulted in: 30

Both operations were performed simultaneously to save time.

   ⏱️  Time: 15.78 seconds

2️⃣ Parallel Execution (max_parallel_tools=4):
I'll make both tool calls at once for you.
Tool #3: slow_calculation

Tool #4: slow_multiplication


DEBUG | strands.types.models.model | finished streaming response from model
DEBUG | strands.tools.executor | tool_count=<2>, tool_executor=<ThreadPoolExecutorWrapper> | executing tools in parallel
DEBUG | strands.handlers.tool_handler | tool=<{'toolUseId': 'tooluse_CqmTgbDpSqCZQN8v8p2GcQ', 'name': 'slow_calculation', 'input': {'x': 10, 'y': 20}}> | invoking
DEBUG | strands.handlers.tool_handler | tool=<{'toolUseId': 'tooluse_3qm3F2tdQJ-hD7qDGfkwfQ', 'name': 'slow_multiplication', 'input': {'x': 5, 'y': 6}}> | invoking
DEBUG | strands.tools.executor | tool_count=<2> | submitted tasks to parallel executor
DEBUG | strands.event_loop.streaming | model=<<strands.models.bedrock.BedrockModel object at 0x000002209BB5A490>> | streaming messages
DEBUG | strands.types.models.model | formatting request
DEBUG | strands.types.models.model | invoking model
DEBUG | strands.types.models.model | got response from model


I've completed both calculations as requested:

1. slow_calculation(x=10, y=20) = 30
2. slow_multiplication(x=5, y=6) = 30

Both operations returned the same result of 30, though they used different mathematical operations to get there (likely addition for

DEBUG | strands.types.models.model | finished streaming response from model
DEBUG | strands.agent.conversation_manager.sliding_window_conversation_manager | window_size=<4>, message_count=<40> | skipping context reduction


 the first and multiplication for the second).   Response: I've completed both calculations as requested:

1. slow_calculation(x=10, y=20) = 30
2. slow_multiplication(x=5, y=6) = 30

Both operations returned the same result of 30, though they used different mathematical operations to get there (likely addition for the first and multiplication for the second).

   ⏱️  Time: 10.92 seconds

⚡ Performance Improvement: 1.4x faster with parallel execution!


## 🎨 Step 7: Custom Agent Loop Components

### Building Your Own Loop Extensions
The agent loop is extensible. Let's create a custom component that adds timing information to each tool execution.

In [10]:
# Create a custom tool wrapper that tracks execution timing
class TimedTool:
    def __init__(self, name, func):
        self.name = name
        self.func = func
        self.execution_times = []
    
    def __call__(self, *args, **kwargs):
        start = time.time()
        result = self.func(*args, **kwargs)
        duration = time.time() - start
        self.execution_times.append(duration)
        return f"{result} (took {duration:.3f}s)"

# Create base mathematical functions
def fibonacci_func(n: int) -> int:
    """Calculate the nth Fibonacci number."""
    if n <= 1:
        return n
    a, b = 0, 1
    for _ in range(2, n + 1):
        a, b = b, a + b
    return b

def factorial_func(n: int) -> int:
    """Calculate the factorial of n."""
    if n <= 1:
        return 1
    result = 1
    for i in range(2, n + 1):
        result *= i
    return result

# Wrap tools with TimedTool to add timing functionality
timed_fibonacci = TimedTool("fibonacci", fibonacci_func)
timed_factorial = TimedTool("factorial", factorial_func)

# Convert to Strands tools
@tool
def fibonacci(n: int) -> str:
    """Calculate the nth Fibonacci number with timing."""
    return timed_fibonacci(n)

@tool
def factorial(n: int) -> str:
    """Calculate the factorial of n with timing."""
    return timed_factorial(n)

# Create agent with timed custom tools
math_agent = Agent(
    model=auth_status.strands_bedrock_model,
    tools=[fibonacci, factorial],
    system_prompt="You are a mathematics assistant. Show your calculations with timing."
)

print("🎨 CUSTOM AGENT LOOP COMPONENTS")
print("=" * 50)

# Complex mathematical request
response = math_agent(
    "Calculate the 10th Fibonacci number and the factorial of 7. "
    "Then explain what these sequences represent."
)

# Analyze the loop execution
print("\n📊 Agent Loop Execution Analysis:")
tool_count = sum(1 for msg in math_agent.messages 
                 for item in msg.get('content', []) 
                 if 'toolUse' in item)
print(f"   Tool invocations: {tool_count}")
print(f"   Total loop iterations: {len(math_agent.messages)}")
print(f"   Final response length: {len(str(response))} characters")

# Show execution times collected by our custom TimedTool
print(f"\n⏱️  Custom Timing Data:")
print(f"   Fibonacci execution times: {timed_fibonacci.execution_times}")
print(f"   Factorial execution times: {timed_factorial.execution_times}")

DEBUG | strands.models.bedrock | config=<{'model_id': 'us.anthropic.claude-3-7-sonnet-20250219-v1:0'}> | initializing
DEBUG | strands.tools.registry | tool_name=<fibonacci> | registering function tool
DEBUG | strands.tools.registry | tool_name=<fibonacci>, tool_type=<function>, is_dynamic=<False> | registering tool
DEBUG | strands.tools.registry | tool_name=<factorial> | registering function tool
DEBUG | strands.tools.registry | tool_name=<factorial>, tool_type=<function>, is_dynamic=<False> | registering tool
DEBUG | strands.tools.registry | tools_dir=<d:\Code Workspace\strands\notebooks\tools> | tools directory not found
DEBUG | strands.tools.registry | tool_modules=<[]> | discovered
DEBUG | strands.tools.registry | tool_count=<0>, success_count=<0> | finished loading tools
DEBUG | strands.tools.registry | tools_dir=<d:\Code Workspace\strands\notebooks\tools> | tools directory not found
DEBUG | strands.tools.registry | getting tool configurations
DEBUG | strands.tools.registry | tool

🎨 CUSTOM AGENT LOOP COMPONENTS
I'll calculate the 10th Fibonacci number and the factorial of 7 for you, and then explain what these sequences represent.
Tool #11: fibonacci


DEBUG | strands.types.models.model | finished streaming response from model
DEBUG | strands.tools.executor | tool_count=<1>, tool_executor=<ThreadPoolExecutorWrapper> | executing tools in parallel
DEBUG | strands.handlers.tool_handler | tool=<{'toolUseId': 'tooluse_GbF6W31PSpCN3hm70FX8lw', 'name': 'fibonacci', 'input': {'n': 10}}> | invoking
DEBUG | strands.tools.executor | tool_count=<1> | submitted tasks to parallel executor
DEBUG | strands.event_loop.streaming | model=<<strands.models.bedrock.BedrockModel object at 0x00000236592DDD00>> | streaming messages
DEBUG | strands.types.models.model | formatting request
DEBUG | strands.types.models.model | invoking model
DEBUG | strands.types.models.model | got response from model



Tool #12: factorial


DEBUG | strands.types.models.model | finished streaming response from model
DEBUG | strands.handlers.tool_handler | tool=<{'toolUseId': 'tooluse_8bJH-_K2SEGSUgfw0jzN0Q', 'name': 'factorial', 'input': {'n': 7}}> | invoking
DEBUG | strands.event_loop.streaming | model=<<strands.models.bedrock.BedrockModel object at 0x00000236592DDD00>> | streaming messages
DEBUG | strands.types.models.model | formatting request
DEBUG | strands.types.models.model | invoking model
DEBUG | strands.types.models.model | got response from model


Now let me explain what these sequences represent:

### Fibonacci Sequence
The Fibonacci sequence is a series of numbers where each number is the sum of the two preceding ones, usually starting with 0 and 1. So the sequence begins: 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, ...

The 10th Fibonacci number is 55, as we calculated above.

The sequence is named after the Italian mathematician Leonardo of Pisa (known as Fibonacci), who introduced it to Western mathematics in his 1202 book "Liber Abaci." It appears frequently in nature, such as in the arrangement of leaves on a stem, the spiral pattern of shells, and the branching of trees.

### Factorial
The factorial of a non-negative integer n (denoted as n!) is the product of all positive integers less than or equal to n. So 7! = 7 × 6 × 5 × 4 × 3 × 2 × 1 = 5040.

Factorials grow very quickly and are important in many areas of mathematics, particularly in combinatorics and probability. They represent the number of ways to arrange n distinct ob

DEBUG | strands.types.models.model | finished streaming response from model
DEBUG | strands.agent.conversation_manager.sliding_window_conversation_manager | window_size=<6>, message_count=<40> | skipping context reduction


 areas like probability theory, statistics, and calculus.
📊 Agent Loop Execution Analysis:
   Tool invocations: 2
   Total loop iterations: 6
   Final response length: 1254 characters

⏱️  Custom Timing Data:
   Fibonacci execution times: [5.7220458984375e-06]
   Factorial execution times: [5.9604644775390625e-06]


## 🎯 Summary: The Agent Loop Architecture

### 🏗️ Key Components We've Explored

1. **Message Processing Pipeline**
   - User input → Model processing → Response generation
   - Automatic conversation history management

2. **Tool Integration Flow**
   - Model decides when to use tools
   - Tool execution (sequential or parallel)
   - Result processing and reasoning
   - Recursive loops for complex tasks

3. **Decision Making Process**
   - AI model analyzes user intent
   - Determines appropriate tools to use
   - Chains multiple operations together
   - Synthesizes final response

4. **Performance Optimizations**
   - Parallel tool execution
   - Efficient message handling
   - Custom extensions for monitoring

### 🚀 What You've Learned

You now understand:
- ✅ How agents process information step-by-step
- ✅ The recursive nature of complex reasoning
- ✅ Tool execution patterns and optimization
- ✅ Debugging and monitoring techniques
- ✅ Custom agent loop extensions

### 💡 Key Insights

1. **The Loop is Recursive**: Agents can call themselves multiple times to complete complex tasks
2. **Tools Enable Intelligence**: The combination of LLMs and tools creates powerful capabilities
3. **Parallelism Matters**: Independent operations can run simultaneously for better performance
4. **Observability is Crucial**: Understanding the loop helps debug and improve agents
5. **Extensibility is Key**: Custom components can enhance the agent loop for specific needs

### 🎯 Next Steps

With this deep understanding of the agent loop, you're ready to:
- Build more sophisticated agents with complex workflows
- Debug agent behavior effectively
- Optimize performance for production use
- Create custom extensions to the agent loop

### 📚 Further Reading

- [Agent Loop Documentation](https://strandsagents.com/0.1.x/user-guide/concepts/agents/agent-loop/)
- [Tool Development Guide](https://strandsagents.com/0.1.x/user-guide/concepts/tools/)
- [Streaming and Callbacks](https://strandsagents.com/0.1.x/user-guide/concepts/streaming/)

### 🎊 Congratulations!

You've mastered the inner workings of the Strands agent loop! This knowledge is fundamental for building advanced AI systems. In the next videos, we'll explore streaming responses, state management, and multi-agent systems.

Remember: The agent loop is the engine that powers intelligent behavior. Understanding it deeply makes you a more effective AI developer! 🚀